**Objetivo**
* Se puede anticipar el rating que va a obtener una película de acuerdo a sus caracteristicas?, análisis mediante clusters jerarquicos.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

# Importa Packages 
import numpy as np                # linear algebra
import pandas as pd               # data frames
import seaborn as sns             # visualizations
import matplotlib.pyplot as plt   # visualizations
import scipy.stats                # statistics
from sklearn import preprocessing
from scipy.cluster.hierarchy import dendrogram, linkage

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

**Análisis de datos**


In [ ]:
# Upload data
movies = pd.read_csv('../input/movies.csv', encoding = "ISO-8859-1")

#Visualización de los 5 primeros registros
print(movies.head())
# informacion de las variables
print(movies.info())
# shape de los datos
print(movies.shape)
# Descripcion de los datos
movies.describe()

Tenemos información sobre 6820 peliculas entre los años 1986 y 2016, nos interesa realizar análisis de closuter sobre las variables numericas;
*  Budget: Presupuesto de una pelicula. Algunas películas no tienen, por lo que aparece como 0.
*  Gross:  Los ingresos de la película
*  Runtime: Tiempo de ejecución: duración de la película
*  Score: Calificación de usuario de IMDb
*  Votes: Número de votos de los usuarios 

In [ ]:
# Compute the correlation matrix
corr=movies.iloc[:,~movies.columns.isin(['company','country','director','genre','name','released',
                                         'star','writer'])].corr()

# Generate a mask for the upper triangle
mask = np.zeros_like(corr, dtype=np.bool)
mask[np.triu_indices_from(mask)] = True

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(11, 9))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(220, 10, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corr, mask=mask, cmap=cmap, center=0,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.show()



Vemos una correlación entre los ingresos de la pelicula y los votos, el presupuesto y los ingresos.

In [ ]:
g = sns.PairGrid(movies.loc[:,["budget","gross","runtime","score","votes","year"]])
g.map_diag(plt.hist, histtype="step", linewidth=3)
g.map_offdiag(plt.scatter)

No se observan grupos diferenciados en las distintas variables, tal vez se puedan formar dos grupos con los votos y las ganacias de las peliculas.

***Análisis de Cluster con el metódo Jerarquico "Single"***

In [ ]:
sin = linkage(movies.loc[:,["gross","score"]],method="single")
#"budget","gross","runtime","score","votes","year"
plt.figure(figsize=(18,6))
dendrogram(sin, leaf_rotation=90)
plt.xlabel('Index')
plt.ylabel('Euclidean distances')
plt.suptitle("DENDROGRAM SINGLE METHOD",fontsize=8)
plt.show()


El dendograma nos indica que no es posible realizar mas de un cluster

In [ ]:
from scipy.cluster.hierarchy import fcluster
movies_clus=movies.copy()

movies_clus['2_clust']=fcluster(sin,2, criterion='maxclust')
movies_clus['3_clust']=fcluster(sin,3, criterion='maxclust')
movies_clus['10_clust']=fcluster(sin,10, criterion='maxclust')
movies_clus.head()

plt.figure(figsize=(24,4))

plt.suptitle("Hierarchical Clustering Single Method",fontsize=18)

plt.subplot(1,3,1)
plt.title("K = 2",fontsize=14)
sns.scatterplot(x="gross",y="votes", data=movies_clus, hue="2_clust")

plt.subplot(1,3,2)
plt.title("K = 3",fontsize=14)
sns.scatterplot(x="gross",y="votes", data=movies_clus, hue="3_clust")

plt.subplot(1,3,3)
plt.title("K = 10",fontsize=14)
sns.scatterplot(x="gross",y="votes", data=movies_clus, hue="10_clust")


In [ ]:
g = sns.PairGrid(movies_clus, vars=["budget","gross","runtime","score","votes","year"], hue='3_clust',palette="RdBu")
g.map(plt.scatter)
g.add_legend()


Definitivamente es obvio que no se pueden realizar clusters sobre el conjunto de datos, con el metodo de "Single".
Intenatremos con "Complete" y depende de los resultados se probara con transformación de datos 

**Análisis de Cluster con el metódo Jerarquico "Complete"**


In [ ]:
com = linkage(movies.loc[:,["gross","score"]],method="complete")
#"budget","gross","runtime","score","votes","year"
plt.figure(figsize=(18,6))
dendrogram(com, leaf_rotation=90)
plt.xlabel('Index')
plt.ylabel('Euclidean distances')
plt.suptitle("DENDROGRAM COMPLETE METHOD",fontsize=8)
plt.show()


In [ ]:
from scipy.cluster.hierarchy import fcluster
movies_clus=movies.copy()

movies_clus['2_clustCom']=fcluster(com,2, criterion='maxclust')
movies_clus['3_clustCom']=fcluster(com,3, criterion='maxclust')
movies_clus['10_clustCom']=fcluster(com,10, criterion='maxclust')
movies_clus.head()

plt.figure(figsize=(24,4))

plt.suptitle("Hierarchical Clustering Complete Method",fontsize=18)

plt.subplot(1,3,1)
plt.title("K = 2",fontsize=14)
sns.scatterplot(x="gross",y="votes", data=movies_clus, hue="2_clustCom")

plt.subplot(1,3,2)
plt.title("K = 3",fontsize=14)
sns.scatterplot(x="gross",y="votes", data=movies_clus, hue="3_clustCom")

plt.subplot(1,3,3)
plt.title("K = 10",fontsize=14)
sns.scatterplot(x="gross",y="votes", data=movies_clus, hue="10_clustCom")


En este caso es mas facil identificar tres grupos entre las peliculas "taquilleras" y "Mejor votadas"

In [ ]:
g = sns.PairGrid(movies_clus, vars=["budget","gross","score","votes"], hue='3_clustCom', palette="RdBu")
g.map(plt.scatter)
g.add_legend()

Podemos identificar grupos: donde las películas más taquilleras tienen mejores votación o las películas con más presupuesto tienen mejor recaudo.


**Conclusión:**
Sin embargo con estas variables no es fácil identificar grupos de películas, se puede decir que entre más taquillera la película mejor votación va a tener, como segundo paso esta explorar realizar estos grupos con variables categóricas por ejemplo: Productora o Director, e identificar el éxito con estas variables.
